In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "tesi2"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/tesi2

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os
import json 


modelName = 'AlexNet'
PRETRAINED = True
n_epochs = 100
batch_size = 8
learning_rate = 1e-3
use_cuda = True

JSON_FILE = 'plotdata.json'


# Associa modelName ad una classe
# senza ancora creare l'oggetto, lo faccio dopo...
models = {
    'AlexNet': models.alexnet,
    'ResNet18': models.resnet18,
    'denseNet121': models.densenet121,
    'vgg16':models.vgg16,
    'googlenet':models.googlenet,
    'inception':models.inception_v3,
    'shufflenet': models.shufflenet_v2_x1_0
}


transforms = transforms.Compose(
[   
    #transforms.RandomVerticalFlip(0.4),
    #transforms.RandomHorizontalFlip(0.4), 
    transforms.ToTensor()
])


def pad_collate(batch):
    X, y = zip(*batch)
    X = list(X)
    maxw, maxh = 0, 0 

    for img in X:
        _, w, h = img.shape
        maxw = max(w, maxw)
        maxh = max(h, maxh)

    for i, img in enumerate(X):
        _, w, h = img.shape
        diff_w = maxw - w
        diff_h = maxh - h
        newImg = F.pad(img, (diff_h, 0, diff_w, 0))
        X[i] = newImg.unsqueeze(0)
    out =  torch.cat(X)
    y   =  torch.tensor(y)    
    return out, y

train_dataset = datasets.ImageFolder(root='roi_train_contrast', transform=transforms)
test_dataset = datasets.ImageFolder(root='roi_test_contrast', transform=transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = pad_collate)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn = pad_collate)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
def imshow(inp, title=None):
    
    inp = inp.cpu() if device else inp
    inp = inp.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
    

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

#images, labels = next(iter(train_dataloader)) 
#print("images-size:", images.shape)

#out = torchvision.utils.make_grid(images)
#print("out-size:", out.shape)

#imshow(out, title=[train_dataset.classes[x] for x in labels])

# Qui creo l'oggetto:
# per crearlo uso models[modelName] che mi dara' la classe,
# e con le parentesi creo l'oggetto, sarebbe equivalente a fare
# models.resnet18(), ma in questo modo e' generale perche' invece
# di models.resnet18 quello prende la classe giusta in base al nome
# che metti in modelName...
# poi gli dico se usare il pretrained o no in base alla
# variabile PRETRAINED che ho definito in alto
net = models[modelName](pretrained=PRETRAINED)
print(net)
net = net.cuda() if device else net

# qui layer finale viene sostituito
# quindi ogni volta
# tocca andare a trovare il nome giusto (dal print(net)
if modelName == 'ResNet18':
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, 2)
    if use_cuda: net.fc = net.fc.cuda()

elif modelName == 'AlexNet':
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, 2)
    if use_cuda: net.classifier[6] = net.classifier[6].cuda()

elif modelName == 'vgg16':
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, 3)
    if use_cuda: net.classifier[6] = net.classifier[6].cuda()

elif modelName == 'denseNet121':
    num_ftrs = net.classifier.in_features
    net.classifier = nn.Linear(num_ftrs, 3)
    if use_cuda: net.classifier = net.classifier.cuda()

elif modelName == 'googlenet':
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, 3)
    if use_cuda: net.fc = net.fc.cuda()

elif modelName == 'inception':
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, 3)
    if use_cuda: net.fc = net.fc.cuda()

elif modelName == 'shufflenet':
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, 3)
    if use_cuda: net.fc = net.fc.cuda()
    
    
    
     

    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_dataloader)

print(net)
PRETRAINEDtxt = 'Pretrained' if PRETRAINED else 'Base'
print(f'Training Model {modelName}{PRETRAINEDtxt}')

for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_dataloader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            #print(outputs_t)
            #torch.argmax(outputs_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(test_dataloader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), f'models/{modelName}{PRETRAINEDtxt}.pt')
            print('Improvement-Detected, save-model')
    net.train()

fig = plt.figure(figsize=(20,10))
plt.title("Train-Validation Accuracy")
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('accuracy', fontsize=12)
plt.legend(loc='best')
plt.savefig(f'plots/Plot{modelName}{PRETRAINEDtxt}.png')


# Salva dati in file JSON (uso sempre lo stesso)
with open(JSON_FILE, 'r') as f:
    jsonfile = json.load(f)
jsonfile[modelName + PRETRAINEDtxt] = {
    'train_acc': train_acc,
    'val_acc': val_acc,
    'train_loss': train_loss,
    'val_loss': val_loss,
}
with open(JSON_FILE, 'w') as f:
    json.dump(jsonfile, f)


def visualize_model(net, num_images=4):
    images_so_far = 0
    fig = plt.figure(figsize=(15, 10))
    
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        outputs = net(inputs)
        _, preds = torch.max(outputs.data, 1)
        preds = preds.cpu().numpy() if use_cuda else preds.numpy()
        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(2, num_images//2, images_so_far)
            ax.axis('off')
            ax.set_title('predictes: {}'.format(test_dataset.classes[preds[j]]))
            imshow(inputs[j])
            
            if images_so_far == num_images:
                return 

plt.ion()
visualize_model(net)
plt.ioff()


cuda:0


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)


KeyboardInterrupt: ignored